In [14]:
import pandas as pd
from src.DBInterface import DBInterface
from bson import json_util
from datetime import datetime


In [15]:
db_interface = DBInterface('mongodb://localhost:27017/?readPreference=primary', 'stocksDB', 'vietStocks')


In [16]:
def to_csv(df: pd.DataFrame) -> None:
    """
    Write a dataframe to a csv file
    :param df: A dataframe
    :return: None
    """
    today = datetime.today().strftime('%Y-%m-%d')
    df.to_csv(f'data/top50_EY_ROE_rankings_{today}.csv', index=False)


# MongoDB way

In [17]:
result = db_interface.collection.aggregate(
   [
    {
        '$match': {
            '$and': [
                {
                    'market_cap': {
                        '$gt': 100
                    }
                }, {
                    'pe': {
                        '$gt': 0
                    }
                }, {
                    'pb': {
                        '$lt': 1, 
                        '$gt': 0
                    }
                }, {
                    '$or': [
                        {
                            'roe': {
                                '$gte': 15
                            }
                        }, {
                            'roa': {
                                '$gte': 15
                            }
                        }
                    ]
                }
            ]
        }
    }, {
        '$addFields': {
            'EY': {
                '$divide': [
                    1, '$pe'
                ]
            }
        }
    }, {
        '$setWindowFields': {
            'partitionBy': None, 
            'sortBy': {
                'EY': -1
            }, 
            'output': {
                'EYRank': {
                    '$denseRank': {}
                }
            }
        }
    }, {
        '$setWindowFields': {
            'partitionBy': None, 
            'sortBy': {
                'roe': -1
            }, 
            'output': {
                'ROERank': {
                    '$denseRank': {}
                }
            }
        }
    }, {
        '$addFields': {
            'finalRank': {
                '$add': [
                    '$EYRank', '$ROERank'
                ]
            }
        }
    }, {
        '$sort': {
            'finalRank': 1
        }
    }, {
        '$project': {
            '_id': 0, 
            'symbol': 1, 
            'company_name': 1, 
            'EY': 1, 
            'roe': 1, 
            'floor': 1, 
            'EYRank': 1, 
            'ROERank': 1, 
            'finalRank': 1
        }
    }
] 
)

# Write to a csv file
df = pd.DataFrame(result)
to_csv(df)

